In [1]:
import sys
sys.path.append('../')
import pnn

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
def gen_pattern_replace_and_matcher_for_loop_pnn_mul(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return reducey_x[redundancy_x_row] * reducey_x[reducey_x_col],\
              non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],\
              reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col]


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      cross_term = torch.sum(cross_term, dim = 2)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [4]:
def gen_pattern_replace_and_matcher_for_loop_pnn_mul_reduce(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim = -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = 2)


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 1)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [5]:
pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],32,[400,400,400],0.1)


In [6]:
pnn_loop = symbolic_trace(pnn_loop)
pnn_loop.graph.print_tabular()

opcode         name                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [46]:
ori , modify = workload_pnn(32,40)

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [26]:
ori.print_readable()

class ProductNeuralNetworkModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        embedding_offsets = self.embedding.offsets
        add = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add);  add = None
        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

'class ProductNeuralNetworkModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        embedding_offsets = self.embedding.offsets\n        add = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add);  add = None\n        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,

In [27]:
modify.print_readable()

class ProductNeuralNetworkModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        offsets = self.offsets
        add_1 = x + offsets;  x = offsets = None
        embed = self.embed(add_1);  add_1 = None
        getitem_2 = embed[(0, slice(None, 40, None))]
        getitem_3 = embed[(slice(None, None, None), slice(40, None, None))]
        getitem_4 = getitem_2[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

'class ProductNeuralNetworkModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        offsets = self.offsets\n        add_1 = x + offsets;  x = offsets = None\n        embed = self.embed(add_1);  add_1 = None\n        getitem_2 = embed[(0, slice(None, 40, None))]\n        getitem_3 = embed[(slice(None, None, None), slice(40, None, None))]\n        getitem_4 = getitem_2[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,

In [ ]:
modify.graph.print_tabular()

In [17]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))


247 ms ± 10.2 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [48]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


891 ms ± 19.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [ ]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


1.03 s ± 13.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [49]:
ori , modify = workload_pnn(128,40)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [50]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.07 s ± 46.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [51]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.46 s ± 41.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [52]:
ori , modify = workload_pnn(128,80)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [53]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.08 s ± 51.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [55]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.59 s ± 18.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [112]:
def gen_pattern_replace_and_matcher_for_loop_pnn_mul(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return reducey_x[redundancy_x_row] * reducey_x[reducey_x_col],\
              non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],\
              reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col]


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      cross_term = torch.sum(cross_term, dim = 2)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [105]:
def workload_pnn_mul(dim,redundancy):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,[400,400,400],0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,[400,400,400],0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [116]:
def workload_pnn_mul_reduce(dim,redundancy):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,[400,400,400],0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,[400,400,400],0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [6]:
ori , modify = workload_pnn_mul_reduce(128,80)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [7]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.2 s ± 107 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [8]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


1.07 s ± 18.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [9]:
ori , modify = workload_pnn_mul_reduce(128,40)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [10]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.51 s ± 41.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [15]:
ori , modify = workload_pnn_mul_reduce(32,40)

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [16]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


966 ms ± 19.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [17]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


734 ms ± 13.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [11]:
def workload_pnn_mul_reduce_avazu(dim,redundancy):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [12]:
ori , modify = workload_pnn_mul_reduce_avazu(128,11)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [13]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


215 ms ± 7.64 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [14]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


147 ms ± 8.22 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [ ]:
def gen_pattern_replace_and_matcher_for_loop_pnn_reduce(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim = -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = 2)

    def pn(self,x):
      cross_term =  x[getitem_node_args[0]] * x[getitem_node_args[1]]
      redundancy_part_pn = cross_term[0,:self.redundancy_cross_part_total]
      non_redundancy_part_pn = cross_term[:,self.redundancy_cross_part_total:self.redundancy_cross_part_total + self.non_redundancy_cross_part_total]
      mixed_part_pn = cross_term[:,self.redundancy_cross_part_total + self.non_redundancy_cross_part_total:]
      return torch.sum(redundancy_part_pn.unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_part_pn,dim = -1),\
              torch.sum(mixed_part_pn,dim = 2)


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(x)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 1)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [76]:
def workload_pnn_reduce(dim,redundancy):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,[400,400,400],0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,[400,400,400],0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [77]:
ori , modify = workload_pnn_reduce(128,80)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [ ]:
modify.print_readable()

In [ ]:
modify

In [80]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.1 s ± 53.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [81]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.05 s ± 26.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [85]:
ori , modify = workload_pnn_reduce(256,80)

now gen workload of pnn with config: dim: 256


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [86]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


5.95 s ± 59.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [87]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


5.98 s ± 112 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [138]:
ori , modify = workload_pnn_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [139]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.02 s ± 43 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [140]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.99 s ± 33.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [ ]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

In [104]:
def gen_pattern_replace_and_matcher_for_loop_pnn(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]

      # cross part linear
      self.redency_cross_part_linear = nn.Linear(self.redency_cross_part_total,self.ori_linear_shape[0],bias = True)
      self.unredency_cross_part_linear = nn.Linear(self.unredency_cross_part_total,self.ori_linear_shape[0],bias = False)
      self.mixed_cross_part_linear = nn.Linear(self.rest_cross_part_total,self.ori_linear_shape[0],bias = False)
      
      # embed part linear 
      self.redency_linear = nn.Linear(self.embed_dim * self.num_prefix,self.ori_linear_shape[0],bias = False)
      self.unredency_linear = nn.Linear(self.embed_dim * self.num_sufix,self.ori_linear_shape[0],bias = False)

    def pn(self,reducey_x,unredency_x):
       redency_x_row, reducey_x_col = list(),list()
       unredency_x_row, unredency_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redency_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             unredency_x_row.append(i),unredency_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redency_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = 2),\
              torch.sum(unredency_x[:,unredency_x_row] * unredency_x[:,unredency_x_col],dim = 2),\
              torch.sum(reducey_x[mixed_x_row] * unredency_x[:,mixed_x_col],dim = 2)


    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_part_embed = self.embed(redency_part)
      unredency_part_embed = self.embed(unredency_part)
      redency_part_pn, unredency_part_pn, mixed_part_pn = self.pn(redency_part_embed,unredency_part_embed)
      
      
      return self.redency_linear(self.embed(redency_part).view(-1,self.embed_dim * self.num_prefix)) + self.unredency_linear(self.embed(unredency_part).view(-1,self.embed_dim * self.num_sufix)) +\
         self.redency_cross_part_linear(redency_part_pn) + self.mixed_cross_part_linear(mixed_part_pn) + self.unredency_cross_part_linear(unredency_part_pn)
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [122]:
def workload_pnn(dim,redundancy,mlp = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [91]:
ori , modify = workload_pnn(128,80)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [92]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.95 s ± 25.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [94]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


798 ms ± 10.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [98]:
def gen_pattern_replace_and_matcher_for_loop_pnn_linear(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.num_fields * self.embed_dim
      # cross part linear
      self.redency_cross_part_linear = nn.Linear(self.redency_cross_part_total,self.ori_linear_shape[0],bias = True)
      self.unredency_cross_part_linear = nn.Linear(self.unredency_cross_part_total,self.ori_linear_shape[0],bias = False)
      self.mixed_cross_part_linear = nn.Linear(self.rest_cross_part_total,self.ori_linear_shape[0],bias = False)
      
      # embed part linear 
      self.redency_linear = nn.Linear(self.embed_dim * self.num_prefix,self.ori_linear_shape[0],bias = False)
      self.unredency_linear = nn.Linear(self.embed_dim * self.num_sufix,self.ori_linear_shape[0],bias = False)

    def pn(self,x):
         cross_term =  x[getitem_node_args[0]] * x[getitem_node_args[1]]
         return torch.sum(cross_term, dim = 2)

    def forward(self,x):
      x = self.embed(x + self.offsets)
      cross_term = self.pn(x)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      len_redundancy = self.embed_dim * self.num_prefix
      len_nonredundancy = len_redundancy + self.embed_dim * self.num_sufix
      len_pn_redundancy = len_nonredundancy + self.redency_cross_part_total
      len_pn_nonredundancy = len_pn_redundancy + self.unredency_cross_part_total
      
      
      
      
      
      return self.redency_linear(x[0,:len_redundancy]) + self.unredency_linear(x[:,len_redundancy : len_nonredundancy]) +\
         self.redency_cross_part_linear(x[0,len_nonredundancy:len_pn_redundancy]) +\
         self.mixed_cross_part_linear(x[:,len_pn_nonredundancy:]) + self.unredency_cross_part_linear(x[:,len_pn_redundancy:len_pn_nonredundancy])
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [103]:
def workload_pnn_linear(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_linear(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [104]:
ori , modify = workload_pnn_linear(128,80)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [105]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.97 s ± 50.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [106]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.96 s ± 47.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [110]:
ori , modify = workload_pnn_linear(32,80,[1000,400,400])

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [111]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


806 ms ± 14 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [112]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


789 ms ± 15.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [135]:
ori , modify = workload_pnn_linear(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [136]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.01 s ± 51 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [137]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.99 s ± 52 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [110]:
def workload_pnn_mul(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [115]:
ori , modify = workload_pnn_mul(32,80,[1000,400,400])

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [116]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


802 ms ± 13.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [117]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


694 ms ± 15.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [144]:
ori , modify = workload_pnn_mul(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [145]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.03 s ± 54.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [146]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.53 s ± 36.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [117]:
def workload_pnn_mul_reduce(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [119]:
ori , modify = workload_pnn_mul_reduce(32,80,[1000,400,400])

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [120]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


810 ms ± 14.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [121]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


289 ms ± 8.55 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [155]:
ori , modify = workload_pnn_mul_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [ ]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.01 s ± 76.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [157]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


935 ms ± 18 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [158]:
ori , modify = workload_pnn(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [159]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


825 ms ± 16.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [123]:
ori , modify = workload_pnn(32,80,[1000,400,400])

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [ ]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


812 ms ± 11.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [125]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


237 ms ± 8.83 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [130]:
def workload_pnn_reduce(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [127]:
ori , modify = workload_pnn_reduce(32,80,[1000,400,400])

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [128]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


803 ms ± 14.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [131]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


810 ms ± 13.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [132]:
ori , modify = workload_pnn_reduce(64,80,[1000,400,400])

now gen workload of pnn with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [133]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


1.53 s ± 19.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [134]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


1.53 s ± 23.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [147]:
ori , modify = workload_pnn_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [142]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.02 s ± 35.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [143]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.99 s ± 35.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [150]:
%timeit -n 1 -r 30 torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long)


4.25 ms ± 1.16 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [148]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 1892.5504684448242 ms
total time: 2302.842378616333 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  mul                             617.649             26.8211
call_function  getitem                         572.174             24.8464
call_function  getitem_1                       559.649             24.3026
call_function  sum_1                            52.7329             2.28991
call_module    mlp_mlp_0                        38.6357             1.67774
call_function  cat                              22.9812             0.997948
call_module    embedding_embedding              18.584              0.807003
call_module    mlp_mlp_2                         2.04372            0.0887479
call_module    mlp_mlp_3                         1.82056            0.0790573
call_module    mlp_mlp_1                         1.33657            0.0580402
call

In [149]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 1783.1566333770752 ms
total time: 2191.3883686065674 ms
Op type        Op           Average runtime (ms)    Pct total runtime
-------------  ---------  ----------------------  -------------------
call_function  mul                   596.477             27.2191
call_function  getitem_1             541.529             24.7117
call_function  getitem               533.669             24.353
call_module    mlp_mlp_0              37.2481             1.69975
call_module    embed                  18.5122             0.844773
call_function  cat                    17.8688             0.815408
call_function  sum_3                  17.4015             0.794084
call_function  concat                  9.47571            0.432407
call_function  sum_2                   2.45595            0.112073
call_module    mlp_mlp_3               2.29788            0.104859
call_method    repeat                  1.15776            0.0528323
call_module    mlp_mlp_2               1.11794            

In [100]:
# 测试对mul算子和reduce算子做单独改写的效果
def gen_pattern_replace_and_matcher_for_loop_pnn_single_mul_reduce(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn1(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return (reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),\
              non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],\
              reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col]

    def pn(self,x):
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn1(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      redundancy_part_pn = cross_term[0,:self.redundancy_cross_part_total]
      non_redundancy_part_pn = cross_term[:,self.redundancy_cross_part_total:self.redundancy_cross_part_total + self.non_redundancy_cross_part_total]
      mixed_part_pn = cross_term[:,self.redundancy_cross_part_total + self.non_redundancy_cross_part_total:]
      return torch.sum(redundancy_part_pn.unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_part_pn,dim = -1),\
              torch.sum(mixed_part_pn,dim = -1)


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(x)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 1)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [97]:
def workload_pnn_single_mul_reduce(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_single_mul_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [127]:
ori , modify = workload_pnn_single_mul_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [128]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.18 s ± 42.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [129]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.68 s ± 27.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [130]:
# 只改了mul
ori , modify = workload_pnn_mul(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [131]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.74 s ± 71.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [132]:
ori , modify = workload_pnn_mul_reduce(128,80,[400,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [133]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


1.02 s ± 18.7 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [21]:
def workload_pnn_loop_optimize(dim,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_loop = symbolic_trace(pnn_loop)
  
  pnn_model_optimize = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1,"innerV2")
  pnn_model_traced_optimize = symbolic_trace(pnn_model_optimize)
  return pnn_model_traced_loop,pnn_model_traced_optimize

In [22]:
loop_model, optimize_model = workload_pnn_loop_optimize(32)

now gen workload of pnn with config: dim: 32


In [24]:
%timeit -n 1 -r 30 with torch.no_grad():loop_model(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


950 ms ± 20.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [25]:
%timeit -n 1 -r 30 with torch.no_grad():optimize_model(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


364 ms ± 5.02 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [43]:
loop_model, optimize_model = workload_pnn_loop_optimize(128)

now gen workload of pnn with config: dim: 128


In [41]:
interp = utils.ProfilingInterpreter(optimize_model)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 386.9314193725586 ms
total time: 395.6868648529053 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  masked_select                   291.617              73.699
call_module    mlp_mlp_0                        26.1631              6.61207
call_function  cat                              22.2235              5.61643
call_module    embedding_embedding              20.6575              5.22068
call_function  bmm                              16.6242              4.20136
call_module    mlp_mlp_2                         1.42789             0.360863
call_module    mlp_mlp_3                         1.24764             0.315311
call_module    mlp_mlp_6                         1.24574             0.314829
call_module    mlp_mlp_5                         1.12343             0.283919
call_module    mlp_mlp_8                         1.09506             0.2767

In [30]:
%timeit -n 1 -r 30 with torch.no_grad():loop_model(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.22 s ± 29.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [31]:
%timeit -n 1 -r 30 with torch.no_grad():optimize_model(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


420 ms ± 5.67 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [32]:
optimize_model.graph.print_tabular()

opcode         name                 target                                                            args                              kwargs
-------------  -------------------  ----------------------------------------------------------------  --------------------------------  ----------
placeholder    x                    x                                                                 ()                                {}
get_attr       embedding_offsets    embedding.offsets                                                 ()                                {}
call_function  add                  <built-in function add>                                           (x, embedding_offsets)            {}
call_module    embedding_embedding  embedding.embedding                                               (add,)                            {}
call_method    transpose            transpose                                                         (embedding_embedding, 1, 2)       {}
call_function  

In [66]:

def gen_pattern_replace_and_matcher_for_optimize(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  # getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.interaction_units = int(num_field * (num_field - 1) / 2)
          self.triu_mask = nn.Parameter(torch.triu(torch.ones(num_field, num_field), 1).bool(),
                                    requires_grad=False) 
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,feature_emb):
        inner_product_matrix = torch.bmm(feature_emb, feature_emb.transpose(1, 2))
        triu_values = torch.masked_select(inner_product_matrix, self.triu_mask)
        return triu_values.view(-1, self.interaction_units)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      self.redundancy_mask = nn.Parameter(torch.triu(torch.ones(self.num_prefix, self.num_prefix), 1).bool(),
                                    requires_grad=False)
      self.non_redundancy_mask = nn.Parameter(torch.triu(torch.ones(self.num_sufix, self.num_sufix), 1).bool(),
                                    requires_grad=False)
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,redundancy_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix

       redundancy_interaction_units = int(prefix * (prefix - 1) / 2)
       non_redundancy_interaction_units = int(sufix * (sufix - 1) / 2)
       mixed_interaction_units = int(prefix * sufix)
       
       non_redundancy_x_t = non_redundancy_x.transpose(1, 2)
       
       redundancy_inner_product_matrix = redundancy_x @ redundancy_x.transpose(-2,-1)
       non_redundancy_inner_product_matrix = torch.bmm(non_redundancy_x, non_redundancy_x_t)
       
       mixed_redundancy_inner_product_matrix = redundancy_x @ non_redundancy_x_t
       
       redundancy_triu_values = torch.masked_select(redundancy_inner_product_matrix, self.redundancy_mask)
       non_redundancy_triu_values = torch.masked_select(non_redundancy_inner_product_matrix,self.non_redundancy_mask)
       
       return redundancy_triu_values.view(-1,redundancy_interaction_units),\
              non_redundancy_triu_values.view(-1,non_redundancy_interaction_units),\
              mixed_redundancy_inner_product_matrix.view(-1,mixed_interaction_units)
    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 2)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [67]:
loop_model, optimize_model = workload_pnn_loop_optimize(128)

now gen workload of pnn with config: dim: 128


In [68]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_optimize(optimize_model,
                                                                      (0,slice(None,40,None)),(slice(None,None,None),slice(40,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)

In [69]:

matches = subgraph_rewriter.replace_pattern_with_filters(optimize_model, pattern, replace,[match])


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [70]:
optimize_model.print_readable()

class ProductNeuralNetworkModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        offsets = self.offsets
        add_1 = x + offsets;  x = offsets = None
        embed = self.embed(add_1);  add_1 = None
        getitem = embed[(0, slice(None, 40, None))]
        getitem_1 = embed[(slice(None, None, None), slice(40, None, None))]
        transpose_1 = getitem_1.transpose(1, 2)
        transpose_2 = getitem.transpose(-2, -1)
        matmul = getitem @ transpose_2;  transpose_2 = None
        bmm_1 = torch.bmm(getitem_1, transpose_1);  getitem_1 = None
        matmul_1 = getitem @ transpose_1;  getitem = transpose_1 = None
        redundancy_mask = self.redundancy_mask
        masked_select_1 = torch.masked_select(matmul, redundancy_mask);  matmul = redundancy_mask = None
        non_redundancy_mask = self.non_redundancy_mask
        masked_select_2 = torch.masked_select(bmm_1, non_redundancy_mask);  bmm_1 = non_redundancy_mask = None
  

'class ProductNeuralNetworkModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        offsets = self.offsets\n        add_1 = x + offsets;  x = offsets = None\n        embed = self.embed(add_1);  add_1 = None\n        getitem = embed[(0, slice(None, 40, None))]\n        getitem_1 = embed[(slice(None, None, None), slice(40, None, None))]\n        transpose_1 = getitem_1.transpose(1, 2)\n        transpose_2 = getitem.transpose(-2, -1)\n        matmul = getitem @ transpose_2;  transpose_2 = None\n        bmm_1 = torch.bmm(getitem_1, transpose_1);  getitem_1 = None\n        matmul_1 = getitem @ transpose_1;  getitem = transpose_1 = None\n        redundancy_mask = self.redundancy_mask\n        masked_select_1 = torch.masked_select(matmul, redundancy_mask);  matmul = redundancy_mask = None\n        non_redundancy_mask = self.non_redundancy_mask\n        masked_select_2 = torch.masked_select(bmm_1, non_redundancy_mask);  bmm_1 = non_redundan

In [71]:
%timeit -n 1 -r 30 with torch.no_grad():optimize_model(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


227 ms ± 10 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [72]:
interp = utils.ProfilingInterpreter(optimize_model)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 208.6329460144043 ms
total time: 218.03569793701172 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  masked_select_1                 101.157              46.3948
call_function  matmul_1                         27.8466             12.7716
call_module    mlp_mlp_0                        23.7854             10.9089
call_module    embed                            21.3883              9.80954
call_function  cat                              16.0255              7.34996
call_function  bmm                               6.16693             2.82841
call_function  concat                            4.93217             2.26209
call_module    mlp_mlp_6                         1.17159             0.537338
call_module    mlp_mlp_8                         1.12987             0.518202
call_module    mlp_mlp_3                         0.809431            0.371238

In [ ]:

class PatternClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = ...
        self.offsets = ...
        self.embed_output_dim = ...

    def pn(self,x):
        return torch.sum(x[cols] * x[rows], dim = 2)
      
    def forward(self,x):
        x = self.embed(x + self.offsets)
        cross_term = self.pn(x)
        return cross_term

In [ ]:
class ReplacementClass(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.offsets = ...
    self.embed = ...

  def pn(self,reducey_x,non_redundancy_x):
      redundancy_x_row, reducey_x_col = list(),list()
      non_redundancy_x_row, non_redundancy_x_col = list(), list()
      mixed_x_row, mixed_x_col = list(), list()
      for i in range(prefix - 1):
        for j in range(i+1,prefix):
            redundancy_x_row.append(i),reducey_x_col.append(j)
      for i in range(sufix - 1):
        for j in range(i + 1,sufix):
            non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

      for i in range(prefix):
        for j in range(sufix):
            mixed_x_row.append(i),mixed_x_col.append(j)
      return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
            torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim = -1),\
            torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = 2)
  def forward(self,x):
    x = self.embed(x + self.offsets)
    redundancy_part_embed = x[redundancy_part_slice]
    non_redundancy_part_embed = x[non_redundancy_part_slice]
    redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
    redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
    cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
    return cross_term

In [ ]:
def single_input_elementwise_op(x):
  return op(x)

In [ ]:
def single_input_elementwise_op_modified(x):
  xr , xn = extractGraph(x)
  result_r = op(xr)
  result_n = op(xn)
  result_r_repeated = result_r.repeat(batch_size,1,1)
  concat_result =  torch.concat([result_r_repeated, result_n], dim = 1)
  return getitem(concat_result,restore_index)

In [ ]:
def dual_input_elementwise_op_without_constant(x1, x2):
  return op(x1, x2)

In [ ]:
def dual_input_elementwise_op_without_constant_modified(x1, x2):
  x1r, x1n, x2r, x2n = extractGraph(x1, x2)
  result_r = op(x1r, x2r)
  result_n = op(x1n,x2n)
  result_r_repeated = result_r.repeat(batch_size,1,1)
  concat_result =  torch.concat([result_r_repeated, result_n], dim = 1)
  return getitem(concat_result,restore_index)

In [ ]:
def dual_input_elementwise_op_with_constant(x1):
  return op(x1)

In [ ]:
def dual_input_elementwise_op_with_constant_modified(x1):
  const_val = op.getConstant()
  
  return op(x1)